In [1]:
import sympy as sym
import math as m
import numpy as np
import pandas as pd

import scipy.optimize
from sympy import pprint
from scipy.optimize import fsolve
from scipy.optimize import least_squares
from scipy.optimize import minimize

import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

import SIC_POVM_functions as sic

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)


[(6+0j), (6+0j), (2.4+0j), (2.5999999999999996+1.637849848627056e-17j), (2.287500258169+0j), (2.4+2.2311252215006176e-17j), (2.4-2.576281494458285e-17j), (2.3000000000000003-4.125283626344832e-18j), (2.587500258169+0j)]


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/optimize/_minpack_py.py:239: ComplexWarning: Casting complex values to real discards the imaginary part
  retval = _minpack._hybrd(func, x0, args, 1, xtol, maxfev,
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/optimize/_minpack_py.py:177: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)


In [17]:
#Class definitions
class seed_sol:
    def __init__(self, seed):
        self.seed = seed
    @property
    def initial_guess(self):
        np.random.seed(self.seed)
        initial_guess = np.random.rand(35)  # random initial guess
        return initial_guess
    @property
    def solution(self):
        solution = fsolve(Lagrange_eqs, self.initial_guess)
        return solution
    @property
    def res_list(self):
        res_list = Lagrange_eqs(self.solution)
        return res_list
    @property
    def res(self):
        res = np.sum(np.abs(self.res_list))
        return res
    def full_POVM(self):
        #break solution into 4 parts, remove the last 7 elements
        sol = self.solution
        solution_add = [sol[:7], sol[7:14], sol[14:21], sol[21:28]]     # divide into 4 parts of 7
        POVM_normalized_9d = [list(i) for i in POVM_normalized_5d]         # deep copy 
        for i in range(2):                              # first two vecs are known
            POVM_normalized_9d[i].extend([0] *4)
        for part in solution_add:                       # adding (already normalized) elements from the solution to make the remaining vectors 9d
            for i in range(7):
                # POVM_normalized_9d[i+2].append(part[i])
                POVM_normalized_9d[i+2].append((1/np.sqrt(6))*(part[i]))        # normalizing coming from the fixing of the lagrangian eqns
        return POVM_normalized_9d


class POVM_relations:
    def __init__(self, POVM_vec_list):
        self.POVM_vec_list = np.array(POVM_vec_list)        # making array for easy dot product
        self.inner_products = {}
    def dot_product(self, i, j):
        if i < 0 or j < 0 or i > 8 or j > 8:
            raise ValueError('Bad index, must be integer 0 to 8')
        return np.vdot(self.POVM_vec_list[i], self.POVM_vec_list[j])
    def all(self, clean = True, threshold = 1e-12, absolutes = True):
        if len(self.POVM_vec_list) != 9:
            raise ValueError('The POVM list must have 9 elements')
        for i in range(9):
            for j in range(i, 9):
                key = f'{i+1}{j+1}'
                value = self.dot_product(i, j)
                self.inner_products[key] = value
        #cleaning the dictionary
        rel = self.inner_products
        for key in rel:
            if absolutes == True:
                rel[key] = abs(rel[key])
            if abs(rel[key]) < threshold:
                rel[key] = 0
        return rel


### Fixing the normalization problem from Lagrange Equations, and associated stuff

In [53]:
# creating five lists POVMs
w = m.e**((2/3)*m.pi*(1j))     # third root of unity
POVM_unnormalized = [[0,1,-1],[-1,0,1],[1,-1,0],[0,w,-w**2],[-1,0,w**2],[1,-w,0],[0,w**2,-w],[-1,0,w],[1,-w**2,0]]             # unnormalized POVM direction vectors
# POVM_vec = (1/(2**.5))*(np.array([[0,1,-1],[-1,0,1],[1,-1,0],[0,w,-w**2],[-1,0,w**2],[1,-w,0],[0,w**2,-w],[-1,0,w],[1,-w**2,0]]))  # normalized POVM direction vectors
POVM_vec = (1/np.sqrt(2))*(np.array([[0,1,-1],[-1,0,1],[1,-1,0],[0,w,-w**2],[-1,0,w**2],[1,-w,0],[0,w**2,-w],[-1,0,w],[1,-w**2,0]]))  # normalized POVM direction vectors

# c4j_list = [2,.5,.5,.5, (-.25-.433013j), -.25, .5, -.25, (-.25-.433013j)]           # fourth elements, not normalized yet. See sic.py for calcs
c4j_list = [2,.5,.5,.5, (-.25-.25j*np.sqrt(3)), (-.25+.25j*np.sqrt(3)), .5, (-.25+.25j*np.sqrt(3)), (-.25-.25j*np.sqrt(3))]           # fourth elements, not normalized yet. See sic.py for calcs
# c5j_list = [0, np.sqrt(15)/2 , 0.38729833462074165, (-0.38729833462074176-0.44721359549995776j), -0.19364916731037082, 0.5809475019311124, -0.38729833462074165, (-0.19364916731037093-0.4472135954999579j), 0.5809475019311124]
c5j_list = [0, np.sqrt(15)/2 , (1/np.sqrt(15))*(3/2), (1/np.sqrt(15))*(2*w**2-.5) ,-(1/np.sqrt(15))*(2+(5/2)*w**2), (1/np.sqrt(15))*(2-w/2), (1/np.sqrt(15))*(2*w-.5), -(1/np.sqrt(15))*(2+(5/2)*w), (1/np.sqrt(15))*(2-.5*w**2)]

POVM_unnormalized_5d = []
for i in range(len(POVM_unnormalized)):
	# Directly append c4j_list[i] and c5j_list[i] to the copies of lists in POVM_unnormalized
	vec_5d_i = POVM_unnormalized[i] + [c4j_list[i], c5j_list[i]]
	POVM_unnormalized_5d.append(vec_5d_i)

POVM_unnormalized_5d[0]    ## unnormalized 5-lists POVM vectors

POVM_normalized_5d = ((1/(6**.5))*(np.array(POVM_unnormalized_5d))).tolist()       # normalized 5-lists POVM vectors

#print and compare
# print(POVM_unnormalized_5d[0])	## UNnormalized 5-lists POVM vectors
# print(POVM_normalized_5d[0])	## normalized 5-lists POVM vectors

# delta_five  = [0.400000000000000, 0.433333333333333, 0.381250043028167, 0.400000000000000, 0.400000000000000, 0.383333333333333, 0.431250043028167]
# delta_five_new = [(0.40000000000000013+0j), (0.4333333333333334-1.5431352694421904e-18j), (0.3812500430281667+0j), (0.40000000000000013-3.0923901048776238e-18j), (0.4+1.898287466587076e-18j), (0.38333333333333347-7.881938927040631e-19j), (0.4312500430281667+3.469446951953614e-18j)]
# print('delta_five_new = ', [np.vdot(POVM_normalized_5d[i], POVM_normalized_5d[i]) for i in range(9)])
delta_five_fixed = [ np.vdot(POVM_normalized_5d[i], POVM_normalized_5d[i]) for i in range(2,9)]
print('delta_five_fixed = ', delta_five_fixed)


delta_five_fixed =  [(0.40000000000000013+0j), (0.4333333333333334-1.5431352694421904e-18j), (0.43333333333333335+1.0728477129103336e-18j), (0.43333333333333346-4.471148278649265e-19j), (0.4333333333333333+2.6275172237666522e-18j), (0.4333333333333335+2.267799535210488e-19j), (0.43333333333333335+6.251554030278836e-19j)]


#### The 1/6 in ortho eq after only the lambda term but not in others is one way of doing it, and prolly not the correct one. Trying this approach for now , later will try the one where 1/6 is multiplied to the coefficients too.

In [55]:
# the main objective function containing the Lagrange equations
def Lagrange_eqs(vars):
    # Unpacking variables, C63 to C99 and y3 to y9
    C63, C64, C65, C66, C67, C68, C69,  C73, C74, C75, C76, C77, C78, C79,  C83, C84, C85, C86, C87, C88, C89,  C93, C94, C95, C96, C97, C98, C99,  y3, y4, y5, y6, y7, y8, y9 = vars
    # global delta_five
    # delta_33, delta_44, delta_55, delta_66, delta_77, delta_88, delta_99 = delta_five
    global delta_five_new
    delta_33, delta_44, delta_55, delta_66, delta_77, delta_88, delta_99 = delta_five_fixed

    # Full set of 28 equations for Cij terms, based on previous interactions
    equations = [
        # Derivatives with respect to C63 to C69
        (1/6)*(np.conj(C63)*y3*(1/6)),
        (1/6)*(np.conj(C63) + np.conj(C64)*y4*(1/6)),
        (1/6)*(np.conj(C63) + np.conj(C64) + np.conj(C65)*y5*(1/6)),
        (1/6)*(np.conj(C63) + np.conj(C64) + np.conj(C65) + np.conj(C66)*y6*(1/6)),
        (1/6)*(np.conj(C63) + np.conj(C64) + np.conj(C65) + np.conj(C66) + np.conj(C67)*y7*(1/6)),
        (1/6)*(np.conj(C63) + np.conj(C64) + np.conj(C65) + np.conj(C66) + np.conj(C67) + np.conj(C68)*y8*(1/6)),
        (1/6)*(np.conj(C63) + np.conj(C64) + np.conj(C65) + np.conj(C66) + np.conj(C67) + np.conj(C68) + np.conj(C69)*y9*(1/6)),
        # Continue with C73 to C79
        # Derivatives with respect to C73 to C79
        (1/6)*(np.conj(C73)*y3*(1/6)), 
        (1/6)*(np.conj(C73) + np.conj(C74)*y4*(1/6)), 
        (1/6)*(np.conj(C73) + np.conj(C74) + np.conj(C75)*y5*(1/6)), 
        (1/6)*(np.conj(C73) + np.conj(C74) + np.conj(C75) + np.conj(C76)*y6*(1/6)),
        (1/6)*(np.conj(C73) + np.conj(C74) + np.conj(C75) + np.conj(C76) + np.conj(C77)*y7*(1/6)),
        (1/6)*(np.conj(C73) + np.conj(C74) + np.conj(C75) + np.conj(C76) + np.conj(C77) + np.conj(C78)*y8*(1/6)),
        (1/6)*(np.conj(C73) + np.conj(C74) + np.conj(C75) + np.conj(C76) + np.conj(C77) + np.conj(C78) + np.conj(C79)*y9*(1/6)),
        # Derivatives with respect to C83 to C89
        (1/6)*(np.conj(C83)*y3*(1/6)), 
        (1/6)*(np.conj(C83) + np.conj(C84)*y4*(1/6)), 
        (1/6)*(np.conj(C83) + np.conj(C84) + np.conj(C85)*y5*(1/6)), 
        (1/6)*(np.conj(C83) + np.conj(C84) + np.conj(C85) + np.conj(C86)*y6*(1/6)),
        (1/6)*(np.conj(C83) + np.conj(C84) + np.conj(C85) + np.conj(C86) + np.conj(C87)*y7*(1/6)),
        (1/6)*(np.conj(C83) + np.conj(C84) + np.conj(C85) + np.conj(C86) + np.conj(C87) + np.conj(C88)*y8*(1/6)),
        (1/6)*(np.conj(C83) + np.conj(C84) + np.conj(C85) + np.conj(C86) + np.conj(C87) + np.conj(C88) + np.conj(C89)*y9*(1/6)),
        # Derivatives with respect to C93 to C99
        (1/6)*(np.conj(C93)*y3*(1/6)),
        (1/6)*(np.conj(C93) + np.conj(C94)*y4*(1/6)), 
        (1/6)*(np.conj(C93) + np.conj(C94) + np.conj(C95)*y5*(1/6)),
        (1/6)*(np.conj(C93) + np.conj(C94) + np.conj(C95) + np.conj(C96)*y6*(1/6)),
        (1/6)*(np.conj(C93) + np.conj(C94) + np.conj(C95) + np.conj(C96) + np.conj(C97)*y7*(1/6)),
        (1/6)*(np.conj(C93) + np.conj(C94) + np.conj(C95) + np.conj(C96) + np.conj(C97) + np.conj(C98)*y8*(1/6)),
        (1/6)*(np.conj(C93) + np.conj(C94) + np.conj(C95) + np.conj(C96) + np.conj(C97) + np.conj(C98) + np.conj(C99)*y9*(1/6)),
        #Now the normalization equations
        (1/6)*(abs(C63)**2 + abs(C73)**2 + abs(C83)**2 + abs(C93)**2) + delta_33 - 1,  
        (1/6)*(abs(C64)**2 + abs(C74)**2 + abs(C84)**2 + abs(C94)**2) + delta_44 - 1,
        (1/6)*(abs(C65)**2 + abs(C75)**2 + abs(C85)**2 + abs(C95)**2) + delta_55 - 1,  
        (1/6)*(abs(C66)**2 + abs(C76)**2 + abs(C86)**2 + abs(C96)**2) + delta_66 - 1,
        (1/6)*(abs(C67)**2 + abs(C77)**2 + abs(C87)**2 + abs(C97)**2) + delta_77 - 1,  
        (1/6)*(abs(C68)**2 + abs(C78)**2 + abs(C88)**2 + abs(C98)**2) + delta_88 - 1,
        (1/6)*(abs(C69)**2 + abs(C79)**2 + abs(C89)**2 + abs(C99)**2) + delta_99 - 1
    ]
    assert len(equations) == 35
    return equations

# Example Usage:



# ig below for np.seed = 2918 
# initial_guess =[0.98030858, 0.35567294, 0.99141023, 0.47111957, 0.34998446, 0.43261049, 0.30442865, 0.21036997, 0.39912775, 0.24321787, 0.91072068, 0.59991195, 0.39894914, 0.9238744 , 0.27633149, 0.9263308 , 0.02004467, 0.09815871, 0.03103966, 0.9009708 , 0.60346855, 0.30131413, 0.22667874, 0.99340599, 0.71874642, 0.71156703, 0.8234102 , 0.80459364, 0.77652217, 0.75734249, 0.06087287, 0.93906535, 0.92258927, 0.22224891, 0.0029041 ] 
initial_guess =[0.13645285, 0.84315108 ,0.12490685 ,0.10271676 ,0.27940578 ,0.98665734, 0.52459312, 0.41561316, 0.70258652, 0.06884098, 0.77664312, 0.84248407, 0.67034251, 0.45412921, 0.69501392, 0.69186904, 0.52676626, 0.45965484, 0.31382016, 0.70279726, 0.29404767, 0.95437514, 0.84051819, 0.87257289, 0.2450531,  0.71094717, 0.34003065, 0.7689673,  0.49019627, 0.6539388, 0.00374957, 0.24125156, 0.29499423, 0.17469819, 0.0711829 ]
print("Initial guess:", initial_guess)
# initial_guess = [0.2] * 15 + [-.2]*15 + [0.1] * 5   # Initial guess for the variables (Cij real and imaginary parts, y)
solution = fsolve(Lagrange_eqs, initial_guess)
residuals = Lagrange_eqs(solution)                   # residuals and residues sum
residuals_sum = np.sum(np.abs(residuals))

print("Solution to the system:", solution)
print("Residuals:", residuals)
print("Residuals sum:", residuals_sum)

Initial guess: [0.13645285, 0.84315108, 0.12490685, 0.10271676, 0.27940578, 0.98665734, 0.52459312, 0.41561316, 0.70258652, 0.06884098, 0.77664312, 0.84248407, 0.67034251, 0.45412921, 0.69501392, 0.69186904, 0.52676626, 0.45965484, 0.31382016, 0.70279726, 0.29404767, 0.95437514, 0.84051819, 0.87257289, 0.2450531, 0.71094717, 0.34003065, 0.7689673, 0.49019627, 0.6539388, 0.00374957, 0.24125156, 0.29499423, 0.17469819, 0.0711829]
Solution to the system: [-1.50824653e+00  1.46575216e+00  1.46575216e+00 -1.46575216e+00
 -1.46575216e+00  1.46575216e+00  1.46575216e+00  3.05920972e-01
 -2.97301745e-01 -2.97301745e-01  2.97301745e-01  2.97301744e-01
 -2.97301744e-01 -2.97301744e-01  9.70985599e-01 -9.43628386e-01
 -9.43628387e-01  9.43628387e-01  9.43628387e-01 -9.43628387e-01
 -9.43628386e-01  5.37393468e-01 -5.22252577e-01 -5.22252578e-01
  5.22252578e-01  5.22252578e-01 -5.22252578e-01 -5.22252578e-01
 -2.65178181e-25  6.17394907e+00  1.73949065e-01  5.82605093e+00
 -1.73949065e-01  6.1739

In [35]:
# for i in range(2800,15000):
for i in [2916,2917,2918, 4531, 4532, 4533, 6032,6033]+ list(range(1, 100)):
    np.random.seed(i)
    initial_guess = np.random.rand(35)
    solution = fsolve(Lagrange_eqs, initial_guess)
    residuals = Lagrange_eqs(solution)                   # residuals and residues sum
    residuals_sum = np.sum(np.abs(residuals))

    if residuals_sum < 1e-10:
        print('\n initial_guess', initial_guess)
        print([i, residuals_sum])

# seed, residue list : [ 2917 4.4191223956293164e-15, 4532 3.3338043426488248e-15, 6033 3.80427932828798e-15, 10998 8.153597002241854e-15, 20990 2.714819746608352e-15, 21860 2.1374692956369812e-15, 25100 3.162636673059558e-15, 28248 6.19787813973094e-16, 28266 8.409742016498353e-15, 30652 3.4325857696401004e-15, 31453 3.041353366217166e-15, 36217 4.054879095452312e-15, 39712 2.1880663663136194e-15, 41193 2.1716813141446073e-15, 45466 8.01171612670997e-15, 46040 5.350693819051618e-15, 46405 1.865265046385399e-15, 52413 2.8128705500231682e-15, 54913 1.9676091484039117e-15, 61595 3.719168940585631e-15, 63488 6.014522363428454e-15, 67336 9.37684050828917e-15, 69405 4.623273839859018e-15, 71313 5.022421141044729e-15, 74069 3.880521317682069e-15, 78313 6.297869491548989e-15, 79895 4.433603719341034e-15, 80937 3.796025080033059e-15, 85854 1.7922593243293907e-15, 86182 9.762464590871239e-15, 86464 4.409168423280531e-15, 87695 3.2460778401093433e-15, 90416 1.482061014995949e-15, 94018 2.833079870204873e-15, 95040 5.246553815378887e-15, 98803 2.314611899961673e-15, 99797 2.929424035866002e-15]
#second method: 56, 6.676816965479679e-16


 initial_guess [0.98419185 0.33341227 0.67370162 0.19639042 0.35444646 0.81336595
 0.24784955 0.45861133 0.87730142 0.39165846 0.32613342 0.09009511
 0.83421775 0.0962116  0.64754508 0.54950399 0.84974054 0.26531732
 0.44086832 0.88088538 0.7363577  0.71264004 0.03567197 0.01973123
 0.32779687 0.56301663 0.74571705 0.72956789 0.99019112 0.96348374
 0.55019533 0.98826167 0.50925462 0.34977244 0.93131972]
[56, 6.676816965479679e-16]

 initial_guess [0.37909853 0.56709817 0.59559251 0.4498589  0.45701983 0.311651
 0.31333736 0.29485946 0.53077246 0.39622202 0.71875037 0.548092
 0.58207508 0.16765094 0.0292772  0.20874192 0.02257544 0.63467444
 0.22444174 0.39830987 0.6195008  0.0019485  0.03566997 0.37678955
 0.21644443 0.40484548 0.86599413 0.25572342 0.60143488 0.22690134
 0.20608795 0.17591994 0.88859933 0.6715823  0.37582037]
[64, 2.0620919034827186e-11]

 initial_guess [0.50066813 0.46806743 0.50078253 0.14917817 0.82027794 0.32136403
 0.779174   0.81508031 0.01480228 0.22757973 0.7

In [8]:
ig2917 = [0.98030858 ,0.35567294, 0.99141023, 0.47111957, 0.34998446 ,0.43261049 , 0.30442865 , 0.21036997 ,0.39912775 ,0.24321787 ,0.91072068, 0.59991195 , 0.39894914 , 0.9238744  ,0.27633149 ,0.9263308 , 0.02004467, 0.09815871 , 0.03103966 , 0.9009708  ,0.60346855 ,0.30131413 ,0.22667874, 0.99340599 , 0.71874642 , 0.71156703 ,0.8234102  ,0.80459364 ,0.77652217, 0.75734249 , 0.06087287 , 0.93906535 ,0.92258927 ,0.22224891 ,0.0029041, ]
sum(np.abs(Lagrange_eqs(ig2917)))

45.24911418718234

In [ ]:
# Example Usage:
for i in range(10):
    np.random.seed(i)
    initial_guess = np.concatenate(([0], np.random.rand(6), [0], np.random.rand(6), [0], np.random.rand(6), [0], np.random.rand(6), np.random.rand(7)))

    solution = fsolve(Lagrange_eqs, initial_guess)
    residuals = Lagrange_eqs(solution)                   # residuals and residues sum
    residuals_sum = np.sum(np.abs(residuals))

    if residuals_sum < 1e-13:
        print("Seed:", i)
        print("Residuals sum:", residuals_sum)
        print("Solution to the system:", solution)

# print("Solution to the system:", solution)
# print("Residuals:", residuals)
# print("Residuals sum:", residuals_sum)

Working. Now, conducting numerous trials using random initial points and filtering out the best outcomes. Then, converting those results into class objects for analysis.

In [ ]:
seed_res_list = []
for i in range(1500):
    np.random.seed(i)
    initial_guess = np.random.rand(35)  # random initial guess
    solution = fsolve(Lagrange_eqs, initial_guess)
    residuals = Lagrange_eqs(solution)
    residuals_sum = np.sum(np.abs(residuals))
    if residuals_sum < 1e-11:
        seed_res_list.append((residuals_sum, i))
    seed_res_list.sort()            # sorted in order of increasing residue sum

seed_list = [i[1] for i in seed_res_list]   # create seed list for later use in class objects
sol_list = [seed_sol(i) for i in seed_list]  # storing class objects, list containing all good solutions as class instances

In [56]:
print(seed_sol(1254).solution, seed_sol(1254).res )  # checking the solution and residue for seed 1254, it is good

[ 2.99985484e-01 -2.91533487e-01 -3.04636688e-01  2.99985484e-01
  2.99985484e-01 -3.04123410e-01 -2.92068892e-01  3.26178983e-01
 -3.16988993e-01 -3.31236312e-01  3.26178983e-01  3.26178983e-01
 -3.30678216e-01 -3.17571147e-01  4.78386813e-01 -4.64908415e-01
 -4.85804088e-01  4.78386813e-01  4.78386813e-01 -4.84985563e-01
 -4.65762225e-01  4.18045498e-01 -4.06267198e-01 -4.24527195e-01
  4.18045498e-01  4.18045498e-01 -4.23811915e-01 -4.07013312e-01
  3.29325384e-28  1.02899151e+00  2.77445120e-02  9.87330081e-01
 -1.26699189e-02  9.98891455e-01 -1.15429795e-03] 1.5776144103944575e-15


In [57]:
povm_1254 = seed_sol(1254).full_POVM()  # checking the full POVM for seed 1254. it is a list of 9 9D vectors, normalized (hopefully)

#checking the inner products
relations_1254 = POVM_relations(povm_1254)    # class object for checking the inner products. has all method, and dot_product method.

In [58]:
print(povm_1254[3],'\n' ,povm_1254[4]  )

# inner product
print(abs(relations_1254.dot_product(2,3)))  

[0j, (-0.20412414523193148+0.35355339059327384j), (0.20412414523193162+0.3535533905932737j), (0.20412414523193154+0j), (-0.15811388300841903-0.18257418583505533j), -0.29153348735392065, -0.3169889931662129, -0.4649084152002779, -0.406267197919503] 
 [(-0.4082482904638631+0j), 0j, (-0.20412414523193162-0.3535533905932737j), (-0.10206207261596577-0.17677681699862874j), (-0.07905694150420949+0j), -0.3046366881250902, -0.3312363115700344, -0.4858040878033379, -0.4245271951136413]
0.5132065491623172


In [59]:
rel = relations_1254.all()  # dictionary of all inner products, cleaned up by default
rel

{'11': 1.0000000000000002,
 '12': 0,
 '13': 0,
 '14': 0,
 '15': 9.936926026465898e-08,
 '16': 0.1443375672974065,
 '17': 0,
 '18': 0.1443375672974065,
 '19': 9.936926027198933e-08,
 '22': 1.0000000000000004,
 '23': 0,
 '24': 0,
 '25': 0.1804219839640731,
 '26': 0,
 '27': 0.1443375672974065,
 '28': 0,
 '29': 0.036084416666666674,
 '33': 0.9999999999999999,
 '34': 0.5132065491623172,
 '35': 0.8101069083609337,
 '36': 0.7147260547463855,
 '37': 0.7147260547463855,
 '38': 0.8087915890668386,
 '39': 0.5166902906428554,
 '44': 1.0,
 '45': 0.42018386597536866,
 '46': 0.8092544935111813,
 '47': 0.6837048860657932,
 '48': 0.7172648364379266,
 '49': 0.6083618168821768,
 '55': 1.0,
 '56': 0.7845103544067316,
 '57': 0.5639429382537904,
 '58': 0.7327973694583723,
 '59': 0.44947345069517197,
 '66': 1.0,
 '67': 0.6414501799308603,
 '68': 0.7844722356848823,
 '69': 0.46353786324629254,
 '77': 0.9999999999999999,
 '78': 0.7838080261733551,
 '79': 0.8099702351879048,
 '88': 1.0,
 '89': 0.421709378287026

In [19]:
sol2917 = seed_sol(2917)
sol2917.initial_guess
sol2917.solution, sol2917.res

(array([ 1.53755984e+00, -1.49423958e+00, -1.56139935e+00, -1.53755984e+00,
        -1.53755984e+00,  1.55876857e+00,  1.49698377e+00, -3.22402747e-01,
         3.13319152e-01,  3.27401526e-01,  3.22402747e-01,  3.22402747e-01,
        -3.26849892e-01, -3.13894566e-01, -1.03425998e+00,  1.00512003e+00,
         1.05029594e+00,  1.03425998e+00,  1.03425998e+00, -1.04852631e+00,
        -1.00696595e+00,  2.49544549e-01, -2.42513710e-01, -2.53413678e-01,
        -2.49544549e-01, -2.49544549e-01,  2.52986705e-01,  2.42959089e-01,
        -1.49681208e-29,  6.17394907e+00,  1.66467072e-01, -5.92398049e+00,
        -1.19239805e+01,  1.76801054e+01,  1.21621768e+01]),
 4.4191223956293164e-15)

In [20]:
print('\n', (1/np.sqrt(6)))


 0.4082482904638631


In [30]:
povm2917 = sol2917.full_POVM()

In [31]:
rel = POVM_relations(povm2917).all()
rel

{'11': 1.0000000000000002,
 '12': 0,
 '13': 0,
 '14': 0,
 '15': 9.936926026465898e-08,
 '16': 0.1443375672974065,
 '17': 0,
 '18': 0.1443375672974065,
 '19': 9.936926027198933e-08,
 '22': 1.0000000000000004,
 '23': 0,
 '24': 0,
 '25': 0.1804219839640731,
 '26': 0,
 '27': 0.1443375672974065,
 '28': 0,
 '29': 0.036084416666666674,
 '33': 1.0000000000000002,
 '34': 0.5132065491623176,
 '35': 0.8101069083609339,
 '36': 0.5204164998665334,
 '37': 0.5204164998665334,
 '38': 0.40929553151898035,
 '39': 0.7075558469242312,
 '44': 1.0000000000000002,
 '45': 0.42018386597536905,
 '46': 0.36070370767703736,
 '47': 0.48395691485547304,
 '48': 0.5082410178128601,
 '49': 0.5597618820714251,
 '55': 1.0000000000000004,
 '56': 0.4346774606048852,
 '57': 0.7076881342476687,
 '58': 0.5384319931654574,
 '59': 0.7369733646388389,
 '66': 1.0000000000000004,
 '67': 0.6414501799308607,
 '68': 0.7844722356848826,
 '69': 0.46353786324629265,
 '77': 1.0000000000000002,
 '78': 0.7838080261733553,
 '79': 0.8099702

In [6]:
sol_56 = seed_sol(56)
sol_56.solution

array([-1.02198257e-01,  9.93188529e-02,  1.03782817e-01,  1.02198257e-01,
       -1.02198257e-01, -1.03607954e-01,  9.95012531e-02, -8.08712387e-01,
        7.85927171e-01,  8.21251283e-01,  8.08712387e-01, -8.08712387e-01,
       -8.19867567e-01,  7.87370535e-01, -6.98241981e-01,  6.78569234e-01,
        7.09068059e-01,  6.98241981e-01, -6.98241981e-01, -7.07873360e-01,
        6.79815434e-01,  1.56460792e+00, -1.52052558e+00, -1.58886679e+00,
       -1.56460792e+00,  1.56460792e+00,  1.58618973e+00, -1.52331804e+00,
       -5.16028852e-28,  6.17394907e+00,  1.66467072e-01, -5.92398049e+00,
        1.19239805e+01,  5.84337836e+00,  1.63085866e-01])

In [15]:
povm56 = sol_56.full_POVM()

# print(povm56[0], '\n', povm56[1])

print(povm56[0][:4], '\n', povm56[7][:4])
print(np.vdot(povm56[0][:3], povm56[7][:3]))
# print(np.vdot(povm56[4], povm56[5]))

rel56 = POVM_relations(povm56)
rel56.all()


[0j, (0.4082482904638631+0j), (-0.4082482904638631+0j), (0.8164965809277261+0j)] 
 [(-0.4082482904638631+0j), 0j, (-0.20412414523193148+0.35355339059327384j), (-0.10206207261596577+0j)]
(0.08333333333333333-0.1443375672974065j)


{'11': 1.0000000000000002,
 '12': 0,
 '13': 0,
 '14': 0,
 '15': 9.936926026465898e-08,
 '16': 0.1443375672974065,
 '17': 0,
 '18': 0.1443375672974065,
 '19': 9.936926027198933e-08,
 '22': 1.0000000000000004,
 '23': 0,
 '24': 0,
 '25': 0.1804219839640731,
 '26': 0,
 '27': 0.1443375672974065,
 '28': 0,
 '29': 0.036084416666666674,
 '33': 1.0000000000000002,
 '34': 0.5132065491623176,
 '35': 0.8101069083609339,
 '36': 0.5204164998665334,
 '37': 0.7147260547463858,
 '38': 0.40929553151898035,
 '39': 0.5166902906428555,
 '44': 1.0000000000000002,
 '45': 0.42018386597536883,
 '46': 0.36070370767703724,
 '47': 0.6837048860657935,
 '48': 0.5082410178128602,
 '49': 0.6083618168821769,
 '55': 1.0,
 '56': 0.43467746060488494,
 '57': 0.5639429382537906,
 '58': 0.5384319931654574,
 '59': 0.449473450695172,
 '66': 1.0000000000000002,
 '67': 0.5928392137277473,
 '68': 0.7844722356848826,
 '69': 0.7519088912701063,
 '77': 1.0000000000000004,
 '78': 0.43423685331037465,
 '79': 0.809970235187905,
 '88':

In [50]:
for i in range(9):
    print(f'{i+1} : ' ,np.round(POVM_unnormalized_5d[i][3:5], 2).tolist())

#checking the inner products of five lists
relxx = POVM_relations(POVM_normalized_5d).all()
relxx

1 :  [2, 0]
2 :  [0.5, 1.94]
3 :  [0.5, 0.39]
4 :  [(0.5+0j), (-0.39-0.45j)]
5 :  [(-0.25-0.43j), (-0.19+0j)]
6 :  [(-0.25+0.43j), (0.58+0j)]
7 :  [0.5, -0.39]
8 :  [(-0.25+0.43j), (-0.19-0.45j)]
9 :  [(-0.25-0.43j), (0.58+0j)]


{'11': 1.0000000000000002,
 '12': 0,
 '13': 0,
 '14': 0,
 '15': 0,
 '16': 0,
 '17': 0,
 '18': 0,
 '19': 0,
 '22': 1.0000000000000004,
 '23': 0,
 '24': 0,
 '25': 0.18042195912175804,
 '26': 0.036084391824351615,
 '27': 0.1443375672974065,
 '28': 0.036084391824351615,
 '29': 0.036084391824351615,
 '33': 0.40000000000000013,
 '34': 0.2,
 '35': 0.20322913997095338,
 '36': 0.20628156324144278,
 '37': 0.17559422921421233,
 '38': 0.20013016597538055,
 '39': 0.2062815632414427,
 '44': 0.4333333333333334,
 '45': 0.1821457749532866,
 '46': 0.23859397170367352,
 '47': 0.10408329997330659,
 '48': 0.22294587036119579,
 '49': 0.13938107236398115,
 '55': 0.38125000000000003,
 '56': 0.2093827734875373,
 '57': 0.1953895681282226,
 '58': 0.17966258885291994,
 '59': 0.14375000000000002,
 '66': 0.43125000000000013,
 '67': 0.18214577495328665,
 '68': 0.15013015186830397,
 '69': 0.21599455047137964,
 '77': 0.4,
 '78': 0.18666480653835105,
 '79': 0.22787514856458863,
 '88': 0.41458333333333347,
 '89': 0.2210

In [56]:
for i in range(9):
    print(f'{i+1} : ' ,np.round(POVM_unnormalized_5d[i][3:5], 2).tolist())

#checking the inner products of five lists
relxx_fixed = POVM_relations(POVM_normalized_5d).all()
relxx_fixed

1 :  [2, 0]
2 :  [0.5, 1.94]
3 :  [0.5, 0.39]
4 :  [(0.5+0j), (-0.39-0.45j)]
5 :  [(-0.25-0.43j), (-0.19+0.56j)]
6 :  [(-0.25+0.43j), (0.58-0.11j)]
7 :  [(0.5+0j), (-0.39+0.45j)]
8 :  [(-0.25+0.43j), (-0.19-0.56j)]
9 :  [(-0.25-0.43j), (0.58+0.11j)]


{'11': 1.0000000000000002,
 '12': 0,
 '13': 0,
 '14': 0,
 '15': 0,
 '16': 0,
 '17': 0,
 '18': 0,
 '19': 0,
 '22': 1.0000000000000004,
 '23': 0,
 '24': 0,
 '25': 0,
 '26': 0,
 '27': 0,
 '28': 0,
 '29': 0,
 '33': 0.40000000000000013,
 '34': 0.2,
 '35': 0.20000000000000004,
 '36': 0.20000000000000007,
 '37': 0.2,
 '38': 0.20000000000000007,
 '39': 0.19999999999999998,
 '44': 0.4333333333333334,
 '45': 0.2333333333333333,
 '46': 0.2333333333333334,
 '47': 0.14529663145135577,
 '48': 0.23333333333333336,
 '49': 0.14529663145135582,
 '55': 0.43333333333333335,
 '56': 0.23333333333333336,
 '57': 0.2333333333333333,
 '58': 0.14529663145135582,
 '59': 0.1452966314513558,
 '66': 0.43333333333333346,
 '67': 0.14529663145135577,
 '68': 0.14529663145135582,
 '69': 0.2333333333333334,
 '77': 0.4333333333333333,
 '78': 0.2333333333333334,
 '79': 0.2333333333333333,
 '88': 0.4333333333333335,
 '89': 0.2333333333333334,
 '99': 0.43333333333333335}